# INTRODUCTION

Let's try to solve the XOR problem. The following codes were based on the ideas from Ian Goodfellow et al. _Deep Learning_ book.

## Using training

The codes written was based in this page's example: 
https://www.tensorflow.org/tutorials/quickstart/beginner


In [1]:
# imports
import tensorflow as tf
import numpy as np

In [2]:
# dataset
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([[0],[1],[1],[0]])
print('X = \n',X)
print('y = \n ',y)

X = 
 [[0 0]
 [0 1]
 [1 0]
 [1 1]]
y = 
  [[0]
 [1]
 [1]
 [0]]


In [3]:
# definition of the model
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(2,activation='relu'),
                                    tf.keras.layers.Dense(1,activation='linear')
])

In [4]:
# prediction without training
prediction_without_training = model(X).numpy()
prediction_without_training

array([[0.        ],
       [0.13101585],
       [0.04067854],
       [0.17169444]], dtype=float32)

In [5]:
# definition of loss function (and one example)
loss_fn = tf.keras.losses.MeanSquaredError()
loss_fn(y,prediction_without_training).numpy()

0.4262275

In [6]:
# configure model for training
model.compile(optimizer=tf.keras.optimizers.SGD(), 
              loss = loss_fn,
              metrics = [tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (4, 2)                    6         
_________________________________________________________________
dense_1 (Dense)              (4, 1)                    3         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [7]:
# adjust parameters to minimize the loss
model.fit(x=X,y=y,epochs=1000)

Epoch 1/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.4262 - mean_squared_error: 0.4262 - mean_absolute_error: 0.5000
Epoch 2/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.4031 - mean_squared_error: 0.4031 - mean_absolute_error: 0.4978
Epoch 3/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.3864 - mean_squared_error: 0.3864 - mean_absolute_error: 0.4977
Epoch 4/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.3741 - mean_squared_error: 0.3741 - mean_absolute_error: 0.4973
Epoch 5/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.3630 - mean_squared_error: 0.3630 - mean_absolute_error: 0.4970
Epoch 6/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.3530 - mean_squared_error: 0.3530 - mean_absolute_error: 0.4966
Epoch 7/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.3441 - mean_squared_error: 0.3441 - mean_absolute_error: 0.4962
Epoch 8/1000
1/1 [================

In [8]:
# test again
prediction_after_training = model(X).numpy()
prediction_after_training

array([[0.2788206 ],
       [0.76935935],
       [0.8854966 ],
       [0.10891791]], dtype=float32)

In [9]:
np.round(prediction_after_training)

array([[0.],
       [1.],
       [1.],
       [0.]], dtype=float32)

In [12]:
# Let's check out the model weights.
# Notice the matrix from the first layer is approximately [[1,1],[1,1]], and
# its bias vector is near [0, -1]. The next layer's weight vector is approx. [1, -2] 
# and its bias value is near 0.

model.trainable_weights

[<tf.Variable 'dense/kernel:0' shape=(2, 2) dtype=float32, numpy=
 array([[0.7537176 , 0.78310925],
        [0.60943186, 0.78185403]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(2,) dtype=float32, numpy=array([ 0.1261915 , -0.78369105], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(2, 1) dtype=float32, numpy=
 array([[ 0.8049115],
        [-1.6218638]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.17724763], dtype=float32)>]

## Using function composition
This was based on functions that can be found here: https://www.tensorflow.org/api_docs/python/tf/linalg


In [68]:
# Weight matrix (from input dimension to hidden layer representation)
W = tf.constant([1,1,1,1],shape=[2,2])
W

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 1],
       [1, 1]], dtype=int32)>

In [69]:
# bias vector in hidden layer representation
c = tf.constant([0,-1],shape=[2,1])
c

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[ 0],
       [-1]], dtype=int32)>

In [71]:
# weight vector (linear transformation from hidden layer representation to output dimension)
w = tf.constant([1,-2],shape=[2,1])
w

# we suppose output dimension's bias is zero (b = 0)

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[ 1],
       [-2]], dtype=int32)>

In [81]:
# to multiply X by W, the X matrix must be turned into an int32 tensor 

matrix_product = tf.linalg.matmul( tf.constant(X,dtype='int32'), W)
matrix_product.numpy()

array([[0, 0],
       [1, 1],
       [1, 1],
       [2, 2]], dtype=int32)

In [85]:
# to addition properly, transpose c and create a 4x2 matrix repeating the rows

sum = tf.math.add(tf.broadcast_to(tf.transpose(c),[4,2]),matrix_product)
sum.numpy()

array([[ 0, -1],
       [ 1,  0],
       [ 1,  0],
       [ 2,  1]], dtype=int32)

In [87]:
# apply the nonlinearity
max_output = tf.math.maximum(sum,0)
max_output.numpy()

array([[0, 0],
       [1, 0],
       [1, 0],
       [2, 1]], dtype=int32)

In [89]:
# apply the linear transformation in output space
f = tf.linalg.matmul(max_output, w)

<tf.Tensor: shape=(4, 1), dtype=int32, numpy=
array([[0],
       [1],
       [1],
       [0]], dtype=int32)>